In [1]:
import pandas as pd
import spacy
import random

In [2]:
#READ NORMALIZED DATA FOR TARGET VERSION OF PIPELINE
df = pd.read_csv("C:\\Users\\brush\\Projekt_NLP\\normalizedResult.tsv", sep="\t")

In [3]:
#READ DATA THAT CONTAINS UNIQUE WORDS FOR EACH TARGET
onlyPerson=[]
onlyGroup=[]
onlyPublic=[]

personFile = open('onlyPersonWords.txt', 'r', encoding="utf-8")
temp = personFile.readlines()
for line in temp:
    onlyPerson.append(line.replace("\n", ""))
    
groupFile = open('onlyGroupWords.txt', 'r', encoding="utf-8")
temp = groupFile.readlines()
for line in temp:
    onlyGroup.append(line.replace("\n", ""))

publicFile = open('onlyPublicWords.txt', 'r', encoding="utf-8")
temp = publicFile.readlines()
for line in temp:
    onlyPublic.append(line.replace("\n", ""))

In [4]:
#CREATE TWO LISTS THAT CONTAIN EACH TEXT OF A TWEET AND ITS TARGET
target = (df.iloc[0:8132 , 2 ].to_list())
text= df.iloc[0:8132, 1 ].to_list()
print (len(text))
print (len(target))

8132
8132


In [5]:
#ADDING ADDITIONAL WORDS WITH CORRESPONDING TARGETS
for x in range(0, 2):
    for word in onlyPerson:
        text.append(word)
        target.append(0)
    for word in onlyGroup:
        text.append(word)
        target.append(1)
    for word in onlyPublic:
        text.append(word)
        target.append(2)
print (len(text))
print (len(target))

39068
39068


In [6]:
#REMOVAL OF TWEETS WITHOUT ANY CONTENT AND ITS CORRESPONDING ENTRY IN TARGET LIST
counter=0
for t in text:
    if type(t) is float:
        text.remove(t)
        target.remove(target[counter])
    counter+=1

print (len(text))
print(len(target))

39017
39017


In [7]:
#TOKENISATION OF TWEETS AND LOWERCASING OF WORDS
array=[]

nlp = spacy.load("de_core_news_sm")
for t in text:
    doc = nlp(t)

    temp = [token.lemma_.lower() for token in doc if token.is_alpha 
#             and not token.pos_ == 'ADJ' 
#             and not token.pos_ == 'CCONJ'
#             and not token.pos_ == 'SCONJ' 
#             and not token.pos_ == 'INTJ'
#             and not token.pos_ == 'PREP'
#             and not token.pos_ == 'ADV'
#             and not token.pos_ == 'VERB'
#               and token.pos_ == "AUX"
#               or token.pos_ == "DET"
#               or token.pos_ == "NOUN"
#               or token.pos_ == "PRON"
#               or token.pos_ == "PROPN"
#               or token.pos_ == "VERB"
#               or token.pos_ == "ADV"
              ]
    array.append((" ".join(temp)))

In [8]:
#RANDOMIZER THAT ZIPS TWO CORRESPONDING LISTS TOGETHER AND SHUFFLES BOTH LISTS SYMMETRICALLY
# temporary = list(zip(text, target))
# random.shuffle(temporary)
# text, target = zip(*temporary)
# text, target = list(text), list(target)

In [9]:
#CREATE DATAFRAME THAT CONTAINS TWEET TEXT AND TARGET
df = pd.DataFrame ({'Text': array , 'Target': target })
#print (df)

In [10]:
#SPLIT DATA INTO TRAIN AND TEST DATA
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df.Text, 
    df.Target, 
    test_size=0.2, # 20% samples will go to test dataset
    random_state=2022, #-> same order of dataset
    stratify=df.Target
)
print("Shape of X_train: ", X_train.shape)
print("Shape of X_test: ", X_test.shape)

Shape of X_train:  (31213,)
Shape of X_test:  (7804,)


In [11]:
y_train.value_counts()

1    12674
0    10949
2     7590
Name: Target, dtype: int64

In [12]:
y_test.value_counts()

1    3169
0    2737
2    1898
Name: Target, dtype: int64

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
import xgboost as xgb

#1. create a pipeline object
clf = Pipeline([
     ('vectorizer_tfidf',TfidfVectorizer()),    
     #('KNN', KNeighborsClassifier())
     ('SVM', SVC()),
     #('rfc', RandomForestClassifier()) 
     #('xgb', xgb.XGBClassifier(objective="binary:logistic", random_state=42)) 
])

#2. fit with X_train and y_train
clf.fit(X_train, y_train)


#3. get the predictions for X_test and store it in y_pred
y_pred = clf.predict(X_test)


#4. print the classfication report
print(classification_report(y_test, y_pred))

# find accuracy scores
accuracy = accuracy_score(y_test, y_pred)
print("The accuracy of prediction is: ", accuracy)

              precision    recall  f1-score   support

           0       0.75      0.80      0.77      2737
           1       0.74      0.86      0.79      3169
           2       0.93      0.60      0.73      1898

    accuracy                           0.77      7804
   macro avg       0.81      0.75      0.77      7804
weighted avg       0.79      0.77      0.77      7804

The accuracy of prediction is:  0.774218349564326


In [14]:
#X_test[:100]
# c =0
# for i in X_test[:200]:
#     print(c,"->", i)
#     c+=1

In [15]:
#y_test[0:50]
# c =0
# for i in y_test[0:]:
#     print(c,"->", i)
#     c+=1

person=0
group=0
public=0
for i in y_test:
    if i == 2:
        public+=1
    if i == 1:
        group+=1
    if i == 0:
        person+=1
print("count of target Person: ", person) 
print("count of target Group: ", group)
print("count of target Public", public)

count of target Person:  2737
count of target Group:  3169
count of target Public 1898


In [16]:
#y_pred[0:50]
# c =0
# for i in y_pred[0:]:
#     print(c,"->", i)
#     c+=1

person=0
group=0
public=0
for i in y_pred:
    if i == 2:
        public+=1
    if i == 1:
        #print(i)
        group+=1
    if i == 0:
        #print(i)
        person+=1
print("count of target Person: ", person) 
print("count of target Group: ", group)
print("count of target Public", public)

count of target Person:  2902
count of target Group:  3669
count of target Public 1233
